In [2]:
# 1. Import Library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import joblib
import os

# 2. Fungsi Klasifikasi Berdasarkan Batasan
def label_kualitas_air(ph, suhu, ammonia):
    bahaya = 0
    waspada = 0

    if ph < 5.0 or ph > 9.0:
        bahaya += 1
    elif 5.0 <= ph <= 6.4 or 8.6 <= ph <= 9.0:
        waspada += 1

    if suhu < 18 or suhu > 32:
        bahaya += 1
    elif 18 <= suhu <= 19 or 31 <= suhu <= 32:
        waspada += 1

    if ammonia > 0.05:
        bahaya += 1
    elif 0.03 <= ammonia <= 0.05:
        waspada += 1

    if bahaya >= 1:
        return "Bahaya"
    elif waspada >= 1:
        return "Waspada"
    else:
        return "Normal"

# 3. Simulasi Dataset
np.random.seed(42)
n = 200
ph = np.random.uniform(4.5, 9.5, n)
suhu = np.random.uniform(15, 35, n)
amonia = np.random.uniform(0, 0.1, n)

df = pd.DataFrame({
    "ph": ph,
    "suhu": suhu,
    "amonia": amonia
})
df["status"] = df.apply(lambda row: label_kualitas_air(row["ph"], row["suhu"], row["amonia"]), axis=1)

# 4. Label Encoding
label_map = {"Bahaya": 0, "Waspada": 1, "Normal": 2}
df["label"] = df["status"].map(label_map)

# 5. Split Data
X = df[["ph", "suhu", "amonia"]].values
y = to_categorical(df["label"].values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Simpan scaler untuk digunakan nanti
os.makedirs("model", exist_ok=True)     
# 7. Buat Model ANN
model = Sequential()
model.add(Dense(16, input_dim=3, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 8. Training
model.fit(X_train_scaled, y_train, epochs=50, batch_size=8, validation_split=0.1)

# 9. Simpan Model
model.save("model/model_ann.h5")

# 10. Evaluasi
score = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Akurasi Uji: {score[1]*100:.2f}%")


c:\Users\NITRO\miniconda3\envs\deeplearning\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 